In [1]:
import json
import numpy as np
import pandas as pd

with open('../georeference.json') as f:
    dd = json.load(f)
    
df_ii = pd.read_csv('../data/inscr.csv')

In [2]:
def janitor(s):
    return (s
            .str.title()
            .str.strip()
            .replace(to_replace=[', ', 'Cabecera Cantonal', 
                                 ' Y ', 'Capital Provincial', 
                                 'De La Republica Del Ecuador'],
                     value='', 
                     regex=True)
           )

df_ii = (df_ii
 .loc[:, ['parroquia_reside', 'canton_reside', 'provincia_reside']]
 .assign(**{c: lambda df_, c=c: df_[c].pipe(janitor) for c in ['parroquia_reside', 'canton_reside', 'provincia_reside']})
)

In [3]:
df_gdd = pd.DataFrame().from_dict(dd, orient='index')

df_gdd = (df_gdd
          .loc[:, ['provincia', 'canton', 'parroquia']])

In [8]:
df_ii.sample(10).T

,345134,559632,716526,507229,337190,889616,46906,649249,77424,403874
parroquia_reside,NaN,NaN,Camilo Ponce Enriquez,Belisario Quevedo,Vuelta Larga,Licto,Tonsupa,Asuncion,NaN,NaN
canton_reside,NaN,NaN,Camilo Ponce Enriquez,Distrito Metropolitano De Quito,Esmeraldas,Riobamba,Atacames,Giron,NaN,NaN
provincia_reside,NaN,NaN,Azuay,Pichincha,Esmeraldas,Chimborazo,Esmeraldas,Azuay,NaN,NaN


In [9]:
df_gdd.sample(10).T

,040452,110651,050152,060256,110456,010169,030155,150452,010750,131351
provincia,Carchi,Loja,Cotopaxi,Chimborazo,Loja,Azuay,Cañar,Napo,Azuay,Manabi
canton,Mira,Espindola,Latacunga,Alausi,Celica,Cuenca,Azogues,El Chaco,San Fernando,Santa Ana
parroquia,Jijon Y Caamaño (Cab. En Rio Blanco),Bellavista,Belisario Quevedo (Guanailin),Pistishi (Nariz Del Diablo),Sabanilla,Turi,Luis Cordero,Linares,San Fernando,Ayacucho


### Cleaning Categoricals

The specific challenge I'm facing is the following:

    Given long pd.Series objects that contain mostly strings and some NaNs and the strings which may well be represented as categories. The problem is that many data points contain typos or else have been altered in the processes of conversion from one encoding to another and they may be correct, but thery are typed differently which prevents merging with another dataset.
    More to the point of what I require is that I need to merge on three columns representing ecuadorian provinces, districts and parishes. That is. I must update the 'provinces', 'district' and 'parish' columns all my dataframe so that their contents match exactly those of counterpart columns in the other dataframe.
    
I have come up with the following idea that maybe if a pd.Series transform to categorical, there may exist a method to change or update these categories. In the fist step, data type conversion, wrong or repeated categories may arise but passing a dictionary to it may run much faster than applying the replace method to the whole pd.Series.

The size of the array is just one problem. The next is the one that mosts confuses me: Which function to apply in order to update array contents. In my head, there exist the following options:
- Write a dictionary with all bad instances as dictionary keys and all the correct or expected words as dictionary values
- Similar to the previous but shorter is to write regex pattern to match bad instances and have them as dict keys with expected words as dict values
- Chat refered to me to the possibility of using measure of words similarity. That given a pair of words, calculate a measure of similarity to discern whether two strings are equivalent.
- Another options is to use a spell checker. As with calculating measure of similarity, there are many libraries available to accomplish this job.

The problem with the last two options is that given the full lenght arrays, they will take much computation power and energy and these processes may even crash.

- Solutions to that problem won't pass the full length arrays to the functions but instead only unique values with which to generate a dictionary for later use with the pd.Series.replace() method. That may not need to transform the variable to  categories in the first place, but this convertion may give a speedup. 

#### Write Custom Dictionary

###### Writing a custom dictionary to apply to pd.Series as-is 

In [26]:
from IPython.display import display

uniqueCombn_ii = df_ii.sort_values(['provincia_reside','canton_reside', 'parroquia_reside']).drop_duplicates().T

with pd.option_context('display.max_columns', 2000):
    display(uniqueCombn_ii)

,292,125206,9155,3430,62089,41839,12154,66,925,1177,27927,28248,15377,162136,7462,3054,29,977,231,495,566,73,946,629,2838,21,891,3127,5557,206,5267,46,148,2065,1962,7265,134,166,1326,423,2634,618,3089,381,13107,5123,941,286,17497,2064,5012,399,10778,33476,5461,6861,2188,4893,10502,6853,19681,42102,10328,4464,11165,34518,2790,32782,2894,23996,1657,3152,1719,3465,6428,3592,325,7809,212858,1881,709,74564,59010,11239,2329,76959,13888,27852,18807,4954,8760,1574,245,2332,2990,9111,24218,4143,521,21237,378,1401,1230,367,228305,15861,2144,18,1460,327,11059,1668,3036,19214,384,1458,11897,5345,801,5327,52916,5919,4214,56246,6150,1996,531,559,5440,98597,4133,6619,35101,2908,2879,45,4389,68497,9985,953,8119,1303,12836,873112,6038,8272,5286,8577,7869,416,2124,6032,16061,6305,234585,1624,6079,3390,8277,552,379,18157,1761,3699,1068,3543,12450,68608,9313,2137,7240,407,88418,3616,7403,16731,876,14598,1683,512,11976,7067,4540,3346,20102,14133,8052,24607,21378,31773,1290,458,11533,1578,10363,165,2900,8279,2832,40078,24507,130786,47243,41677,4379,2181,22727,2350,3768,360,12047,172,46063,4763,13858,3982,2074,2373,1297,3256,1745,10991,307666,2965,2015,551011,55,12688,32912,23369,4768,112615,2184,96878,4213,1016,195064,5034,5342,36984,7964,92,1263,58208,4681,6705,211,1000,3078,5293,13058,450637,2755,8162,233,56,1543,1312,5613,5682,168,21382,17810,8934,2014,836,41,235,2718,749,743,34603,14354,14307,1741,3594,1058,3908,1466,1883,26990,17581,6260,11935,1479,26871,341,4966,1033,3194,14,2,3597,4272,26,9723,46427,12377,2576,256,5092,16689,41224,1265,594,20217,7957,413,34952,85297,79600,1654,12151,51512,1540,2571,5288,534,1193,7861,5307,3877,4197,5972,1041,113725,1725,630,25813,20206,5209,893,18285,51980,42,6242,91,20,147,220,9451,19034,2271,218,4473,15860,2178,4909,158,2163,3835,2060,17489,8226,14199,7639,11841,6124,13183,76,21605,19846,80088,2797,811,172498,231872,11791,57,5898,5190,12977,1014,6228,19496,140,10631,4947,8425,11337,22213,15352,36431,8656,7055,43534,7633,2022,6047,37498,8506,2166,1556,19277,12872,508,4770,7442,170078,12222,196629,10967,103009,25050,27479,27778,8134,5802,693,331,75,20599,9051,10922,127,10702,6716,53,308,31975,9734,671,45120,1579,78781,476,119974,87,4017,414,10176,73770,2948,1932,638,610,3254,76330,15251,4160,486,823,2437,428535,18036,2381,90569,1315,54701,36664,14821,1491,91813,94196,1111,186699,13727,474745,11052,1528,10582,875,124514,274,3998,67,9112,670,2061,180,74,43872,1752,546,1203,2474,899,4836,11157,7938,17521,2747,58,35,8253,7383,653,100,1786,2641,228,195,59,477,1194,4116,156,9596,2446,7976,71,2112,2019,6872,312,5358,7,129,146,10,3201,3755,1266,1376,1584,563,5547,261,511,370,3160,317,208,2039,61724,19401,579,7270,3052,863,179,3529,115,748,1909,4993,1062,2840,16787,202,1037,529,1189,1372,2891,42028,4961,1042,1931,5520,36343,24709,3406,27,405,3951,2449,1882,27144,3625,1480,5272,12535,7725,2573,82,26105,28520,5225,2956,2689,191908,3687,37667,3176,216,389,2311,6321,23904,105,604,1472,130,60399,7022,3209,587,720,7204,71574,4481,48,1110,547,992,7353,17492,3555,3789,19212,9573,63706,3728,3643,5366,3812,862,1952,1822,11950,917,7007,4320,1087,1084,379790,3032,5398,3761,1356,52013,2152,3013,18801,23680,10632,828,43354,2237,3929,6250,58934,4001,3795,3776,5652,1544,13994,647,34796,3116,573,2759,8264,8411,304,2666,72538,193,293,2000,11892,3608,14567,107,34006,51,1221,881,3023,2568,10406,866,20236,1510,101108,22411,4752,27090,51306,3436,10435,295,282,12556,4908,19239,17018,12512,58680,1015,497,12432,3320,18481,31527,4758,12228,2250,10001,832,35314,6839,20984,42598,1160,64250,428,12092,37334,517,42717,20398,49358,20584,805052,5015,32310,81399,586,3589,3487,17446,776,34628,595,5801,614,1935,9524,738,1793,2331,14640,9022,784,1003,816,184,9993,249,2479,473,11655,2104,1402,1188,2650,5838,1646,121,981,499,152,1628,4326,449,3376,270,1005,9818,1112,64,20481,2642,575,1724,1499,296,2669,5946,32,2531,15497,3026,1818,278,3567,920,14379,3652,5455,234,18534,187,1735,3298,3898,5702,455,724,24579,323,2026,26705,1236,2171,23813,675,2961,195901,15

In [28]:
from IPython.display import display

uniqueCombn_gdd = df_gdd.sort_values(['provincia','canton', 'parroquia']).drop_duplicates().T

with pd.option_context('display.max_columns', 2000):
    display(uniqueCombn_gdd)

,011550,011150,011152,011153,011151,011154,010151,010153,010154,010155,010150,010152,010156,010157,010158,010159,010160,010161,010162,010163,010164,010165,010166,010167,010168,010169,010170,010171,011250,011253,010251,010250,010252,011450,010352,010350,010353,010359,010354,010356,010357,010360,010358,010451,010452,010453,010450,011050,011051,010552,010553,010562,010554,010556,010550,010559,010561,010650,010652,010751,010750,010851,010852,010850,010853,011351,011352,011350,010951,010953,010952,010954,010955,010956,010950,020650,020250,020251,020351,020353,020350,020354,020355,020450,020151,020150,020153,020155,020156,020160,020157,020158,020159,020750,020551,020552,020553,020550,020554,020556,020555,040250,040251,040252,040253,040255,040254,040350,040351,040352,040353,040451,040452,040453,040450,040552,040551,040553,040554,040555,040550,040650,040651,040151,040159,040153,040154,040155,040161,040156,040157,040150,040158,030150,030151,030153,030154,030155,030156,030157,030158,030160,030250,030254,030251,030252,030253,030350,030351,030352,030363,030353,030354,030355,030356,030357,030358,030362,030361,030650,030651,030550,030450,030451,030452,030750,060251,060250,060253,060254,060255,060256,060257,060258,060259,060260,060450,060551,060550,060552,060553,060554,060351,060352,060353,060354,060350,061050,060651,060650,060652,060751,060750,060752,060753,060754,060755,060756,060757,060758,060759,060850,060956,060951,060955,060952,060950,060953,060954,060151,060152,060153,060154,060155,060156,060157,060158,060159,060150,060160,060161,050251,050250,050252,050157,050151,050152,050153,050154,050150,050156,050158,050159,050161,050162,050350,050351,050352,050353,050451,050453,050455,050456,050450,050457,050458,050551,050552,050553,050554,050555,050550,050651,050652,050653,050650,050751,050752,050753,050754,050750,070250,070255,070251,070254,070351,070352,070353,070350,070354,070355,070450,070451,070550,070651,070650,070652,070654,070653,070750,071452,071451,071450,071453,070152,070150,070851,070850,070951,070952,070956,070953,070950,070954,070955,071051,071052,071053,071054,071050,071055,071056,071151,071152,071150,071153,071257,071251,071252,071253,071254,071250,071255,071256,071351,071352,071353,071354,071355,071356,071357,071359,071358,071350,080650,080651,080652,080653,080654,080251,080252,080253,080262,080254,080255,080256,080257,080258,080263,080259,080260,080261,080264,080250,080152,080154,080153,080150,080159,080163,080165,080166,080168,080850,080351,080352,080353,080350,080354,080355,080356,080357,080358,080452,080451,080455,080453,080450,080454,080751,080752,080753,080754,080750,080755,080555,080551,080552,080553,080554,080556,080557,080558,080550,080559,080560,080561,080562,200250,200251,200151,200152,200150,200351,200350,200352,090250,090350,090450,090550,090551,092350,090650,090652,090653,090654,090656,090750,090852,090851,090850,090950,092750,090150,090152,090153,090156,090157,090158,092850,092450,091051,091053,091050,091054,091151,091150,091152,091153,091154,091250,092550,091350,091450,091452,091451,092150,091950,091951,091953,091952,091650,091651,092053,092050,092056,092055,091850,092251,092250,100250,100251,100252,100253,100254,100357,100351,100350,100352,100353,100354,100355,100356,100358,100151,100152,100153,100154,100155,100156,100157,100150,100451,100452,100453,100450,100454,100455,100456,100457,100458,100459,100551,100552,100550,100553,100651,100652,100653,100654,100655,100650,110250,110251,110252,110254,110253,110350,110351,110352,110353,110354,110450,110451,110455,110456,110457,110554,110551,110550,110552,110553,110654,110650,110651,110656,110655,110652,110653,110751,110750,110753,110754,110756,110151,110152,110153,110154,110155,110150,110156,110163,110157,110158,110159,110160,110161,110162,110852,110851,110850,110853,111651,111650,110951,110958,110950,110952,110954,110956,110957,110959,111452,111451,111450,111050,111051,111052,111053,111054,111055,111551,111550,111552,111151,111152,111153,111154,111155,111156,111157,11

###### Update or change the categories in a cateogical variable

In [11]:
data_parr_cat = data_parr.astype('category')

In [13]:
data_parr_cat.cat.rename_categories?

Signature: data_parr_cat.cat.rename_categories(*args, **kwargs)
Docstring:
Rename categories.

Parameters
----------
new_categories : list-like, dict-like or callable

    New categories which will replace old categories.

    * list-like: all items must be unique and the number of items in
      the new categories must match the existing number of categories.

    * dict-like: specifies a mapping from
      old categories to new. Categories not contained in the mapping
      are passed through and extra categories in the mapping are
      ignored.

    * callable : a callable that is called on all items in the old
      categories and whose return values comprise the new categories.

inplace : bool, default False
    Whether or not to rename the categories inplace or return a copy of
    this categorical with renamed categories.

    .. deprecated:: 1.3.0

Returns
-------
cat : Categorical or None
    Categorical with removed categories or None if ``inplace=True``.

Raises
------
Valu

#### Write Custom Cictionary That Uses Regex Patterns 

#### Using measures of similarity
Important note: Don't pass the full-length array to function like cosine_similarity from sklearn but instead use the pd.Series.apply() method for each of the strings in the array or else (and probably better) get all unique values with the `pd.Series.unique()` method and generate a dictionary to be used with replace. Yet another interesting alternative is to convert `pd.Series` to categorical with `astype('category')` and use an existing method to update or modify categories.

In [ ]:
import jellyfish

# at least three functions to use
for name, func in {'Damerau-Levenshtein' : jellyfish.damerau_levenshtein_distance, 'Hamming' : jellyfish.hamming_distance, 'Levenshtein': jellyfish.levenshtein_distance}.items():
    print(f'{name} :', func('Distrito Metropolitano de Quito', 'Distrito Metropolitano Quito'))

#### Using A Spell Checker